In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import os

In [ ]:
#gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop3-2.2.5.jar lib/gcs-connector-hadoop3-2.2.5.jar

In [ ]:
# Modify spark environment template to fix host name for dockerization
!cp $SPARK_HOME/conf/spark-env.sh.template $SPARK_HOME/conf/spark-env.sh && echo "SPARK_MASTER_HOST=127.0.0.1" >> $SPARK_HOME/conf/spark-env.sh

In [2]:
# Start Local Standalone cluster
!cd $SPARK_HOME && ./sbin/start-master.sh

starting org.apache.spark.deploy.master.Master, logging to /home/jdelzio/spark/spark-3.5.1-bin-hadoop3/logs/spark-jdelzio-org.apache.spark.deploy.master.Master-1-de-zoomcamp-deb.us-west2-a.c.intricate-reef-411403.internal.out


In [3]:
# Start cluster worker
!cd $SPARK_HOME && ./sbin/start-worker.sh spark://127.0.0.1:7077

starting org.apache.spark.deploy.worker.Worker, logging to /home/jdelzio/spark/spark-3.5.1-bin-hadoop3/logs/spark-jdelzio-org.apache.spark.deploy.worker.Worker-1-de-zoomcamp-deb.us-west2-a.c.intricate-reef-411403.internal.out


In [4]:
# set up spark configuration to GCP
PROJECT_HOME = os.getenv("HOME")+"/data-engineering-zoomcamp/project" # this may need to be updated when dockerized
credentials_location = PROJECT_HOME+"/.google/credentials/gcp.json"

conf = SparkConf() \
    .setMaster("spark://127.0.0.1:7077") \
    .setAppName("process_raw_data") \
    .set("spark.jars", PROJECT_HOME+"/lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [5]:
# set up spark context
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/04/13 06:46:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
# Start Spark session using standalone cluster
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [7]:
# Pull data from GCS Bucket into spark df
df_test = spark.read.parquet('gs://test_bucket-intricate-reef-41103/raw/*')

In [8]:
df_test.count()

113935

In [9]:
spark.stop()

In [10]:
# Stop Local Standalone cluster
!cd $SPARK_HOME && ./sbin/start-master.sh

org.apache.spark.deploy.master.Master running as process 28937.  Stop it first.


In [11]:
# Stop Worker
!cd $SPARK_HOME && ./sbin/stop-worker.sh

stopping org.apache.spark.deploy.worker.Worker
